In [21]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import scipy.stats as stats

In [22]:
house = pd.read_parquet('./data/clean_house.parquet')
house

,id,date,price,bedrooms,bathrooms,sqft_living,sqft_lot,floors,waterfront,view,...,lat,long,sqft_living15,sqft_lot15,month,grade_value,has_view,renovated,yr_since_renovate,total_rooms
0,7129300520,2014-10-13,221900.0,3,1.00,1180,5650,1.0,0,0.0,...,47.5112,-122.257,1340,5650,10,7,0,0.0,60.0,3.00
1,6414100192,2014-12-09,538000.0,3,2.25,2570,7242,2.0,0,0.0,...,47.7210,-122.319,1690,7639,12,7,0,1.0,24.0,6.75
2,5631500400,2015-02-25,180000.0,2,1.00,770,10000,1.0,0,0.0,...,47.7379,-122.233,2720,8062,2,6,0,0.0,82.0,2.00
3,2487200875,2014-12-09,604000.0,4,3.00,1960,5000,1.0,0,0.0,...,47.5208,-122.393,1360,5000,12,7,0,0.0,50.0,12.00
4,1954400510,2015-02-18,510000.0,3,2.00,1680,8080,1.0,0,0.0,...,47.6168,-122.045,1800,7503,2,8,0,0.0,28.0,6.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
21592,263000018,2014-05-21,360000.0,3,2.50,1530,1131,3.0,0,0.0,...,47.6993,-122.346,1530,1509,5,8,0,0.0,6.0,7.50
21593,6600060120,2015-02-23,400000.0,4,2.50,2310,5813,2.0,0,0.0,...,47.5107,-122.362,1830,7200,2,8,0,0.0,1.0,10.00
21594,1523300141,2014-06-23,402101.0,2,0.75,1020,1350,2.0,0,0.0,...,47.5944,-122.299,1020,2007,6,7,0,0.0,6.0,1.50
21595,291310100,2015-01-16,400000.0,3,2.50,1600,2388,2.0,0,0.0,...,47.5345,-122.069,1410,1287,1,8,0,0.0,11.0,7.50


In [23]:
expensive = house[house.price >= 750000]
cheaper = house[house.price < 750000]

In [24]:
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
import sklearn.preprocessing as sk
from sklearn.compose import make_column_transformer

In [25]:
y_exp = expensive.price
y_exp_log = np.log(expensive.price)

In [63]:
y_cheap = cheaper.price
y_cheap_log = np.log(cheaper.price)

In [26]:
# R2 = .8158
trainsimple = ['zipcode','sqft_living']

In [27]:
# R2 = .8435
trainsimple1 = ['zipcode','sqft_living','grade_value']

In [28]:
# R2 = .8527
trainlight = ['zipcode','sqft_living','grade_value','waterfront']

In [29]:
# R2 = .8620
trainlight2 = ['zipcode','sqft_living','grade_value','waterfront','view']

In [30]:
# R2 = .8621 - bathrooms didn't change R2 at all
trainmedium = ['zipcode','sqft_living','grade_value','waterfront','view','bedrooms']

In [31]:
# R2 = .8662
trainmedium2 = ['zipcode','sqft_living','grade_value','waterfront','view','bedrooms','condition']

In [32]:
# R2 = .8669
trainmedium3 = ['zipcode','sqft_living','grade_value','waterfront','view','bedrooms','condition','yr_since_renovate']

In [33]:
# R2 = .8732
trainheavy = ['zipcode','sqft_living','grade_value','waterfront','view','bedrooms','condition','yr_since_renovate','sqft_lot']

In [34]:
# R2 = .8748
trainheavy2 = ['zipcode','sqft_living','grade_value','waterfront','view','bedrooms','condition','yr_since_renovate','sqft_lot', 'sqft_living15']

In [35]:
# R2 = .8773
trainheavy3 = ['zipcode','sqft_living','grade_value','waterfront','view','bedrooms','condition','yr_since_renovate','sqft_lot', 'sqft_living15', 'sqft_above']

In [36]:
# R2 = .8784
trainheavy4 = ['zipcode','sqft_living','grade_value','waterfront','view','bedrooms','condition','yr_since_renovate','sqft_lot', 'sqft_living15', 'sqft_above','total_rooms']

In [61]:
# R2 = .8811
trainall = cheaper.drop(['id','date','grade','price'],axis=1)
colsx = list(trainall.columns)

In [58]:
X_exp = expensive[trainsimple]

In [64]:
X_cheap = cheaper[colsx]

In [65]:
model = LinearRegression()
X_train, X_test, y_train, y_test = train_test_split(X_cheap,y_cheap_log,test_size =.33, random_state = 42)

In [66]:
ohe = sk.OneHotEncoder()
tranformed_train = ohe.fit_transform(X_train[['zipcode']])
X_train[ohe.categories_[0]] = tranformed_train.toarray()
transformed_test = ohe.fit_transform(X_test[['zipcode']])
X_test[ohe.categories_[0]] = transformed_test.toarray()

# transformer_train = make_column_transformer((ohe, ['zipcode']),remainder='passthrough')
# transformed_train = transformer_train.fit_transform(X_train)

# transformer_train = make_column_transformer((ohe, ['has_view']),remainder='passthrough')
# transformed_train = transformer_train.fit_transform(X_train)

# X_train = pd.DataFrame(transformed_train,columns=transformer_train.get_feature_names())

# transformer_test = make_column_transformer((ohe, ['zipcode']),remainder='passthrough')
# transformed_test = transformer_test.transform(X_test)

# transformer_test = make_column_transformer((ohe, ['has_view']),remainder='passthrough')
# transformed_test = transformer_test.transform(X_test)

# X_test = pd.DataFrame(transformed_test,columns=transformer_test.get_feature_names())

X_train.drop(['zipcode',98103],axis=1)
X_test.drop(['zipcode',98103],axis=1)

X_train['sqft_living'] = np.log(X_train.sqft_living)
X_test['sqft_living'] = np.log(X_test.sqft_living)

X_train['sqft_living15'] = np.log(X_train.sqft_living15)
X_test['sqft_living15'] = np.log(X_test.sqft_living15)

X_train['sqft_lot'] = np.log(X_train.sqft_lot)
X_test['sqft_lot'] = np.log(X_test.sqft_lot)

X_train['sqft_lot15'] = np.log(X_train.sqft_lot15)
X_test['sqft_lot15'] = np.log(X_test.sqft_lot15)

X_train['sqft_above'] = np.log(X_train.sqft_above)
X_test['sqft_above'] = np.log(X_test.sqft_above)

X_train['sqft_basement'] = np.log(1 + X_train.sqft_basement)
X_test['sqft_basement'] = np.log(1 + X_test.sqft_basement)

model.fit(X_train, y_train)
predictions = model.predict(X_test)
r2 = r2_score(y_test, predictions)
rmse = mean_squared_error(y_test, predictions, squared=False)
print(r2,rmse)

0.8063147925840108 0.17079445114879332


C:\Users\capta\anaconda3\lib\site-packages\sklearn\utils\validation.py:1688: FutureWarning: Feature names only support names that are all strings. Got feature names with dtypes: ['int64', 'str']. An error will be raised in 1.2.
  warnings.warn(
C:\Users\capta\anaconda3\lib\site-packages\sklearn\utils\validation.py:1688: FutureWarning: Feature names only support names that are all strings. Got feature names with dtypes: ['int64', 'str']. An error will be raised in 1.2.
  warnings.warn(
